# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [29]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [30]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [31]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [32]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [33]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [34]:
# Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS music
        WITH REPLICATION = 
        {'class' : 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print (e)


#### Set Keyspace

In [35]:
# Set KEYSPACE to the keyspace specified above

try : 
    session.set_keyspace('music')
except Exception as e :
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [36]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_music_table_1 = "CREATE TABLE IF NOT EXISTS artist_table"
create_music_table_1 = create_music_table_1 + (""" (session_id INT, item_in_session INT, artist TEXT, song_title TEXT,
                                        song_length FLOAT, PRIMARY KEY (session_id, item_in_session)
                )""")
try:
    session.execute(create_music_table_1)
except Exception as e:
    print(e)
    
select_query1 = """ SELECT artist, song_title, song_length FROM artist_table
                    WHERE session_id = %s 
                    AND item_in_session = %s 
                    """
                    

In [37]:
# The below code its help set up the CSV file to open it. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#         print(line) 
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_table(session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        # the below listed columns is the whole colums which found in event_datafile_new.csv file. amd 
        #it will help us to check the array or the location of our column which we created in the table 
# ['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']


In [38]:
# to use the appropriate datatypes refer the following link (https://docs.datastax.com/en/cql/3.3/cql/cql_reference/cql_data_types_c.html)

#### Do a SELECT to verify that the data have been inserted into each table

In [39]:
## the below code are the SELECT statement to verify the data was populated into the table
try:
    rows = session.execute(select_query1, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print("\n","Artist: "+row.artist, "\nSong Title: "+row.song_title,   "\nSong Length: "+str(row.song_length)," \n ")
    


 Artist: Faithless 
Song Title: Music Matters (Mark Knight Dub) 
Song Length: 495.30731201171875  
 


In [41]:
## 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

 
'''                                         
                                        very important note:
 The combination of the PARTITION KEY alone or with the addition of CLUSTERING COLUMNS should be used appropriately to uniquely identify 
 each row. So According to our data set and our scenario it’s not recommends using PRIMARY KEY (userId, sessionId, itemInSession) 
 for performance perspective. it degrades the performance it use only userId as partition because sessions belonging to the same user 
 might be in different nodes. This will cause a performance issue if the database is very large. Therefore, we should use both userId 
 and sessionId as partition keys so sessions from the same user are stored together. so, in our senario we can you use as the following PRIMARY KEY ((userId, sessionId), itemInSession) rather 
than PRIMARY KEY (userId, sessionId, itemInSession) 
For more info : https://docs.datastax.com/en/dse/5.1/cql/cql/cql_using/whereClustering.html

'''



create_music_table2 = "CREATE TABLE IF NOT EXISTS user_table"
create_music_table2 = create_music_table2 + (""" (user_id INT, session_id INT, item_in_session INT, 
                                        artist_name TEXT, song_title TEXT, user_first_name TEXT, 
                                        user_last_name TEXT, PRIMARY KEY ((user_id, session_id), item_in_session)
                )""")

try :
    session.execute(create_music_table2) 
except Exception as e  :
    print (e)
select_query2 = """ SELECT artist_name, song_title, user_first_name, user_last_name FROM user_table
                    WHERE user_id = %s
                    AND session_id = %s 
                    """
    
                    

In [ ]:
# Insertion for Table 2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_table(user_id, session_id, item_in_session, artist_name, \
                                            song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
## SELECT statement for Table 2
try:
    rows = session.execute(select_query2, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist:- "+row.artist_name, "\nSong Title:- "+row.song_title, "\nUser First Name:- "+row.user_first_name, "\nUser Last Name:- "+row.user_last_name," \n")

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

create_table3 = "CREATE TABLE IF NOT EXISTS song_table"
create_table3 = create_table3 + (""" (song_title TEXT, user_id INT, user_first_name TEXT, user_last_name TEXT, 
                                        PRIMARY KEY ((song_title), user_id)
                )""")
try:
    session.execute(create_table3)
except Exception as e:
    print(e)
    
    
                    

In [ ]:
selectquery3 = """ SELECT song_title,song_title,user_id, user_first_name, user_last_name FROM song_table WHERE song_title = %s;"""


In [28]:
# Insertion for Table 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_table (song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

InvalidRequest: Error from server: code=2200 [Invalid query] message="unconfigured table song_table"

In [14]:

# session.execute(selectquery3,'All Hands Against His Own')

try:
    rows = session.execute(selectquery3, ('All Hands Against His Own',))
    
except Exception as e:
    print(e)
    

for row in rows:
    print("song title :-"+row.song_title,"\nUser First Name:- "+row.user_first_name, "\nUser Last Name:- "+row.user_last_name ," \n ")

    

name 'selectquery3' is not defined


### Drop the tables before closing out the sessions

In [4]:
## Drop the table before closing out the sessions

In [139]:
drop_query1 = "drop table artist_table"
try:
    rows = session.execute(drop_query1)
except Exception as e:
    print(e)
    
drop_query2 = "drop table user_table"
try:
    rows = session.execute(drop_query2)
except Exception as e:
    print(e)
    
drop_query3 = "drop table song_table"
try:
    rows = session.execute(drop_query3)
except Exception as e:
    print(e)
    
    

### Close the session and cluster connection¶

In [140]:
session.shutdown()
cluster.shutdown()